# Jorge - Test notebook

## Data processing

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import warnings
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

train = pd.read_csv('../data/train.csv',header=0)
test = pd.read_csv('../data/test.csv',header=0)

/var/folders/09/t9n5xk9x22j8tr_9772wtssc0000gn/T/ipykernel_48751/3893326307.py:9: DtypeWarning: Columns (435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('../data/train.csv',header=0)


In [2]:
def pre_processing_nn(df, train = True):
    df["time"] = pd.to_datetime(df["time"], format='%H:%M:%S')
    df['time_min'] = df['time'].dt.hour * 60 + df['time'].dt.minute
    
    def actual_time(df, col_var):
        df[['hours', 'minutes']] = df[col_var].str.extract('(\d+):(\d+)')
        df['time_diff'] = df['hours'].astype(int) * 60 - df['minutes'].astype(int)
        
        df['actual_time'] = df['time_min'] + df['time_diff']
            
        df['radians'] = (df["actual_time"] / 1440) * (2 * np.pi)
        
        df['sin_time'] = np.sin(df['radians'])
        df['cos_time'] = np.cos(df['radians'])
        
        df.drop(columns=['radians', 'hours', 'minutes', 'time_diff'], inplace=True)
        
        return df

    variables = ['bg', 'ins', 'carbs', 'cals', 'hr', 'steps', 'activity']
    
    X_df = pd.DataFrame()
    
    for var in variables:
        cols = [col for col in df.columns if col.startswith(var)]
        var_df = pd.melt(df, id_vars=['id', 'time_min'], value_vars=cols, var_name='time', value_name=var)

        if var == 'bg':
            target_scaler = MinMaxScaler()
            var_df[var] = target_scaler.fit_transform(var_df[[var]])
            
            if train:
                Y = var_df[var_df['time'] == 'bg+1:00']
                Y = Y[["id",var]]
                Y = Y.sort_values('id').reset_index(drop=True)[var]
            
                var_df = var_df[var_df['time'] != 'bg+1:00']
            
        elif var == 'activity':
            label_encoder = LabelEncoder()
            var_df[var] = label_encoder.fit_transform(var_df[var].fillna("missing"))
            var_df[var] = var_df[var] + 1
            var_df[var] = var_df[var].replace(23, np.nan)

        else:
            scaler = MinMaxScaler()
            var_df[var] = scaler.fit_transform(var_df[[var]])

        var_df = actual_time(var_df, 'time')
        var_df = var_df.drop(["time_min", "time"], axis = 1)
        
        if X_df.empty:
            X_df = var_df.copy()
        else:
            X_df = X_df.merge(var_df, on=['id', 'actual_time', 'sin_time', 'cos_time'], how='left')

    X_df = X_df.fillna(-0.01)
    X_df = X_df.sort_values('id').reset_index(drop=True)
    
    descriptor_layers = {}
    for id_key, group in X_df.groupby('id'):
        group.drop(columns=['id'], inplace=True)
        group.set_index('actual_time', inplace=True)
        descriptor_layers[id_key] = group
    
    arrays = []
    for id_key, group_df in descriptor_layers.items():
        arrays.append(group_df.values)

    X = np.stack(arrays)
    
    if train:
        return X, Y, target_scaler
    else:
        return X, target_scaler

In [3]:
X_tmp, Y_tmp, target_scaler = pre_processing_nn(pd.read_csv('../data/train.csv',header=0))

/var/folders/09/t9n5xk9x22j8tr_9772wtssc0000gn/T/ipykernel_48751/1463098824.py:1: DtypeWarning: Columns (435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506) have mixed types. Specify dtype option on import or set low_memory=False.
  X_tmp, Y_tmp, target_scaler = pre_processing_nn(pd.read_csv('../data/train.csv',header=0))


In [4]:
print(X_tmp.shape)
print(Y_tmp.shape)

(177024, 72, 9)
(177024,)


In [5]:
X_test, target_scaler = pre_processing_nn(test, train = False)

## Neural networks models

In [6]:
import numpy as np
from pandas import read_csv
from sklearn import preprocessing
import pandas as pd

from tensorflow import keras

# Tensorflow and keras import
from keras import layers, models, utils, callbacks, optimizers

# # Utils to manipulate data
from keras.utils import to_categorical

# # Keras models
from keras.models import Sequential

# # Keras layers
from keras.layers import Dense, Dropout

from keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D, ConvLSTM2D
from keras.layers import Flatten, Input, Add


# # Plotting
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import GRU, Dense, Masking, LSTM, Bidirectional, TimeDistributed, RepeatVector
from keras.utils import plot_model
from keras.models import Sequential
from keras.layers import GRU, Dense, Masking, Dropout, BatchNormalization
from keras import regularizers
from keras.callbacks import EarlyStopping


### Multi-step models

In [8]:
n_steps, n_features, n_outputs = X_tmp.shape[1], X_tmp.shape[2], 12

In [9]:
# Y_pred_0 = Y_pred[:, -1]

# test_ids = test['id']
# #Create a DataFrame for submission
# submission_df = pd.DataFrame({
#     'id': test_ids,  
#     'bg+1:00': Y_pred_0
# })

# print(submission_df.head(10))

#### LSTM

In [10]:
model_LSTM = Sequential()
model_LSTM.add(Masking(mask_value=-0.01, input_shape=(n_steps, n_features)))
model_LSTM.add(BatchNormalization())

model_LSTM.add(LSTM(128, activation='relu', input_shape=(n_steps, n_features)))
model_LSTM.add(Dropout(0.2))

model_LSTM.add(Dense(64, activation='relu'))

model_LSTM.add(Dense(n_outputs))

model_LSTM.compile(optimizer='adam', loss='mse')
model_LSTM.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model_LSTM.fit(X_tmp, Y_tmp, epochs=50, 
          batch_size=1000, 
          validation_split=0.3, 
          callbacks=[early_stopping])

/opt/miniconda3/envs/aiao-env/lib/python3.9/site-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/opt/miniconda3/envs/aiao-env/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking (Masking)               │ (None, 72, 9)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 72, 9)          │            36 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │        70,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 12)             │           780 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 79,728 (311.44 KB)

 Trainable params: 79,710 (311.37 KB)

 Non-trainable params: 18 (72.00 B)

Epoch 1/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 49s 386ms/step - loss: 0.0239 - val_loss: 0.0188
Epoch 2/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 52s 418ms/step - loss: 0.0119 - val_loss: 0.0143
Epoch 3/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 52s 417ms/step - loss: 0.0111 - val_loss: 0.0126
Epoch 4/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 52s 419ms/step - loss: 0.0116 - val_loss: 0.0121
Epoch 5/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 52s 421ms/step - loss: 0.0109 - val_loss: 0.0120
Epoch 6/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 54s 433ms/step - loss: 0.0103 - val_loss: 0.0169
Epoch 7/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 52s 423ms/step - loss: 0.0100 - val_loss: 0.0136
Epoch 8/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 53s 426ms/step - loss: 0.0097 - val_loss: 2.1991
Epoch 9/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 52s 419ms/step - loss: 0.0096 - val_loss: 1208.6575
Epoch 10/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 53s 428ms/step - loss: 0.0105 - val_loss: 0.0123


In [11]:
Y_pred_LSTM = model_LSTM.predict(X_test)
print(Y_pred_LSTM.shape)
Y_pred_LSTM = Y_pred_LSTM.reshape(3644, 12)
Y_pred_LSTM = target_scaler.inverse_transform(Y_pred_LSTM)
print(Y_pred_LSTM.shape)

print(Y_pred_LSTM[:, -1])

114/114 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step
(3644, 12)
(3644, 12)
[7.988031  7.1104307 7.1441526 ... 6.5859833 7.301219  6.8103447]


#### Encoder-Decoder LSTM

The model will not output a vector sequence directly. The model will be comprised of two sub models, the encoder to read and decode the input sequence, and the decoder that will read the encoded input sequence and make a one-step prediction for each element in the output sequence

The `TimeDistributed` allows the LSTM decoder to figure out the context required for each step in the output sequence and the wrapped dense layers to interpret each time step separately, yet reusinf the same weights to perform the interpretation

The network outputs a three-dimesional vector with the same structure as the input: `[samples, timesteps, features]`

In [12]:
model_LSTM_ED = Sequential()
model_LSTM_ED.add(Masking(mask_value=-0.01, input_shape=(n_steps, n_features)))
model_LSTM_ED.add(BatchNormalization())

model_LSTM_ED.add(LSTM(128, activation = "relu", input_shape=(n_steps, n_features)))
model_LSTM_ED.add(Dropout(0.3))
model_LSTM_ED.add(RepeatVector(n_outputs))

model_LSTM_ED. add(LSTM(128, activation = "relu", return_sequences=True))
model_LSTM_ED.add(Dropout(0.3))

model_LSTM_ED.add(TimeDistributed(Dense(100, activation='relu')))
model_LSTM_ED.add(TimeDistributed(Dense(1)))

model_LSTM_ED.compile(optimizer='adam', loss='mse')
model_LSTM_ED.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model_LSTM_ED.fit(X_tmp, Y_tmp, epochs=50,
            batch_size=1000,
            validation_split=0.3,
            callbacks=[early_stopping])

/opt/miniconda3/envs/aiao-env/lib/python3.9/site-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/opt/miniconda3/envs/aiao-env/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking_1 (Masking)             │ (None, 72, 9)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 72, 9)          │            36 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │        70,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_vector (RepeatVector)    │ (None, 12, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 12, 128)        │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 12, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 12, 100)        │        12,900 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 12, 1)          │           101 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 215,277 (840.93 KB)

 Trainable params: 215,259 (840.86 KB)

 Non-trainable params: 18 (72.00 B)

Epoch 1/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 67s 523ms/step - loss: 0.0187 - val_loss: 0.0150
Epoch 2/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 64s 513ms/step - loss: 0.0118 - val_loss: 0.0133
Epoch 3/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 64s 518ms/step - loss: 0.0124 - val_loss: 0.0121
Epoch 4/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 65s 521ms/step - loss: 0.0112 - val_loss: 0.0120
Epoch 5/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 65s 526ms/step - loss: 49246476.0000 - val_loss: 7914.0161
Epoch 6/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 64s 515ms/step - loss: 388428192.0000 - val_loss: 0.2029
Epoch 7/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 63s 512ms/step - loss: 0.0125 - val_loss: 0.9574
Epoch 8/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 63s 511ms/step - loss: 0.0123 - val_loss: 1.5382
Epoch 9/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 67s 542ms/step - loss: 0.0119 - val_loss: 0.7969


In [13]:
Y_pred_LSTM_ED = model_LSTM_ED.predict(X_test)
print(Y_pred_LSTM_ED.shape)
Y_pred_LSTM_ED = Y_pred_LSTM_ED.reshape(3644, 12)
Y_pred_LSTM_ED = target_scaler.inverse_transform(Y_pred_LSTM_ED)
print(Y_pred_LSTM_ED.shape)

114/114 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step
(3644, 12, 1)
(3644, 12)


##### CNN-LSTM Encoder-Decoder

In [14]:
n_steps, n_features, n_outputs = X_tmp.shape[1], X_tmp.shape[2], 12

model_CNN_LSTM = Sequential()
model_CNN_LSTM.add(Masking(mask_value=-0.01, input_shape=(n_steps, n_features)))

model_CNN_LSTM.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_steps, n_features)))
model_CNN_LSTM.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model_CNN_LSTM.add(MaxPooling1D(pool_size=2))

model_CNN_LSTM.add(Flatten())

model_CNN_LSTM.add(RepeatVector(n_outputs))
model_CNN_LSTM.add(LSTM(128, activation='relu', return_sequences=True))

model_CNN_LSTM.add(TimeDistributed(Dense(100, activation='relu')))
model_CNN_LSTM.add(TimeDistributed(Dense(1)))

model_CNN_LSTM.compile(optimizer='adam', loss='mse')
model_CNN_LSTM.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model_CNN_LSTM.fit(X_tmp, Y_tmp, epochs=50,
            batch_size=1000,
            validation_split=0.3,
            callbacks=[early_stopping])

/opt/miniconda3/envs/aiao-env/lib/python3.9/site-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/opt/miniconda3/envs/aiao-env/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/opt/miniconda3/envs/aiao-env/lib/python3.9/site-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'conv1d' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.w

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking_2 (Masking)             │ (None, 72, 9)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 70, 64)         │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 68, 64)         │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 34, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2176)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_vector_1 (RepeatVector)  │ (None, 12, 2176)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 12, 128)        │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 12, 100)        │        12,900 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, 12, 1)          │           101 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,207,305 (4.61 MB)

 Trainable params: 1,207,305 (4.61 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50


/opt/miniconda3/envs/aiao-env/lib/python3.9/site-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'conv1d' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


124/124 ━━━━━━━━━━━━━━━━━━━━ 65s 516ms/step - loss: 0.1338 - val_loss: 0.0123
Epoch 2/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 70s 564ms/step - loss: 0.0128 - val_loss: 0.0114
Epoch 3/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 66s 534ms/step - loss: 0.0115 - val_loss: 0.0107
Epoch 4/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 66s 534ms/step - loss: 0.0108 - val_loss: 0.0104
Epoch 5/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 64s 517ms/step - loss: 0.0105 - val_loss: 0.0112
Epoch 6/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 64s 518ms/step - loss: 0.0103 - val_loss: 0.0112
Epoch 7/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 64s 514ms/step - loss: 0.0101 - val_loss: 0.0104
Epoch 8/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 63s 509ms/step - loss: 0.0098 - val_loss: 0.0106
Epoch 9/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 64s 516ms/step - loss: 0.0097 - val_loss: 0.0104
Epoch 10/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 64s 516ms/step - loss: 0.0095 - val_loss: 0.0101
Epoch 11/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 64s 514ms/step - loss: 0.0094 - val_loss: 0.0102
Epoch 12/50
124/124 ━━━━━━━━━━

In [15]:
Y_pred_CNN_LSTM = model_CNN_LSTM.predict(X_test)
print(Y_pred_CNN_LSTM.shape)
Y_pred_CNN_LSTM = Y_pred_CNN_LSTM.reshape(3644, 12)
Y_pred_CNN_LSTM = target_scaler.inverse_transform(Y_pred_CNN_LSTM)
print(Y_pred_CNN_LSTM.shape)

print(Y_pred_CNN_LSTM[:, -1])

114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
(3644, 12, 1)
(3644, 12)
[8.544109 8.408361 8.469498 ... 8.51095  8.329508 8.40501 ]


##### ConvLSTM Encoder-Decoder

In [16]:
# model_ConvLSTM = Sequential()
# model_ConvLSTM.add(Masking(mask_value=-0.01, input_shape=(n_steps, n_features)))

# model_ConvLSTM.add(ConvLSTM2D(filters = 64,
#                               kernel_size = (1, 3),
#                               activation = "relu",
#                               input_shape = (n_steps, 1, n_features, 1)))
# model_ConvLSTM.add(Dropout(0.3))

# model_ConvLSTM.add(Flatten())
# model_ConvLSTM.add(RepeatVector(n_outputs))
# model_ConvLSTM.add(LSTM(128, activation = "relu", return_sequences = True))
# model_ConvLSTM.add(Dropout(0.3))

# model_ConvLSTM.add(TimeDistributed(Dense(100, activation='relu')))
# model_ConvLSTM.add(TimeDistributed(Dense(1)))

# model_ConvLSTM.compile(optimizer='adam', loss='mse')
# model_ConvLSTM.summary()

# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# model_ConvLSTM.fit(X_tmp, Y_tmp, epochs=50,
#             batch_size=1000,
#             validation_split=0.3,
#             callbacks=[early_stopping])

##### ResLSTM

### Single-step models

In [17]:
n_steps, n_features = 72, 9

#### LSTM vanilla

In [18]:
model_LSTM_single = Sequential()
model_LSTM_single.add(Masking(mask_value=-0.01, input_shape=(n_steps, n_features)))
model_LSTM_single.add(BatchNormalization())

model_LSTM_single.add(LSTM(128, activation='relu', input_shape=(n_steps, n_features)))
model_LSTM_single.add(Dropout(0.2))

model_LSTM_single.add(Dense(64, activation='relu'))

model_LSTM_single.add(Dense(1))

model_LSTM_single.compile(optimizer='adam', loss='mse')
model_LSTM_single.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model_LSTM_single.fit(X_tmp, Y_tmp, epochs=50, 
          batch_size=1000, 
          validation_split=0.3, 
          callbacks=[early_stopping])

/opt/miniconda3/envs/aiao-env/lib/python3.9/site-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/opt/miniconda3/envs/aiao-env/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking_3 (Masking)             │ (None, 72, 9)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 72, 9)          │            36 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 128)            │        70,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 79,013 (308.64 KB)

 Trainable params: 78,995 (308.57 KB)

 Non-trainable params: 18 (72.00 B)

Epoch 1/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 52s 411ms/step - loss: 0.0212 - val_loss: 0.0161
Epoch 2/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 59s 473ms/step - loss: 0.0118 - val_loss: 0.0139
Epoch 3/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 56s 448ms/step - loss: 1.9146 - val_loss: 0.0126
Epoch 4/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 55s 443ms/step - loss: 0.0120 - val_loss: 0.0121
Epoch 5/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 53s 425ms/step - loss: 0.6299 - val_loss: 0.0119
Epoch 6/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 52s 416ms/step - loss: 0.0121 - val_loss: 0.0118
Epoch 7/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 56s 454ms/step - loss: 0.0120 - val_loss: 0.0123
Epoch 8/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 56s 451ms/step - loss: 0.0119 - val_loss: 0.0121
Epoch 9/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 56s 456ms/step - loss: 0.0119 - val_loss: 0.0121
Epoch 10/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 55s 447ms/step - loss: 0.0116 - val_loss: 0.0121
Epoch 11/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 56s 453ms/step - loss: 0.0117 - val_loss: 0.0120


In [19]:
Y_pred_LSTM_single = model_LSTM_single.predict(X_test)
print(Y_pred_LSTM_single.shape)
# Y_pred_LSTM_single = Y_pred_LSTM_single.reshape(3644, 12)
Y_pred_LSTM_single = target_scaler.inverse_transform(Y_pred_LSTM_single)
print(Y_pred_LSTM_single.shape)

print(Y_pred_LSTM_single[:, -1])

114/114 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step
(3644, 1)
(3644, 1)
[34.500725 10.08062   8.488428 ...  9.319648 12.337003 10.174983]


#### LSTM stacked

In [20]:
model_LSTM_single = Sequential()
model_LSTM_single.add(Masking(mask_value=-0.01, input_shape=(n_steps, n_features)))
model_LSTM_single.add(BatchNormalization())

model_LSTM_single.add(LSTM(128, activation='relu', input_shape=(n_steps, n_features), return_sequences=True))
model_LSTM_single.add(Dropout(0.2))

model_LSTM_single.add(LSTM(128, activation='relu'))
model_LSTM_single.add(Dense(64, activation='relu'))

model_LSTM_single.add(Dense(1))

model_LSTM_single.compile(optimizer='adam', loss='mse')
model_LSTM_single.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model_LSTM_single.fit(X_tmp, Y_tmp, epochs=50, 
          batch_size=1000, 
          validation_split=0.3, 
          callbacks=[early_stopping])

/opt/miniconda3/envs/aiao-env/lib/python3.9/site-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/opt/miniconda3/envs/aiao-env/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking_4 (Masking)             │ (None, 72, 9)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 72, 9)          │            36 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 72, 128)        │        70,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 72, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 210,597 (822.64 KB)

 Trainable params: 210,579 (822.57 KB)

 Non-trainable params: 18 (72.00 B)

Epoch 1/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 135s 1s/step - loss: 0.0172 - val_loss: 0.0173
Epoch 2/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 134s 1s/step - loss: 0.0111 - val_loss: 0.0156
Epoch 3/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 136s 1s/step - loss: 0.0104 - val_loss: 0.0131
Epoch 4/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 136s 1s/step - loss: 0.0101 - val_loss: 0.0126
Epoch 5/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 135s 1s/step - loss: 0.0097 - val_loss: 0.0122
Epoch 6/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 133s 1s/step - loss: 0.0091 - val_loss: 0.0134
Epoch 7/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 133s 1s/step - loss: 0.0087 - val_loss: 0.7204
Epoch 8/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 132s 1s/step - loss: 0.0085 - val_loss: 0.0376
Epoch 9/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 131s 1s/step - loss: 0.0084 - val_loss: 6071.6826
Epoch 10/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 132s 1s/step - loss: 1522.6090 - val_loss: 0.0144


#### Bidirectional LSTM

In [21]:
model_bidirectional = Sequential()
model_bidirectional.add(Masking(mask_value=0., input_shape=(n_steps, n_features)))

model_bidirectional.add(BatchNormalization())
model_bidirectional.add(Bidirectional(LSTM(64, activation = "relu", 
                             input_shape=(n_steps, n_features), return_sequences = True)))

model_bidirectional.add(LSTM(64, activation = "relu"))

model_bidirectional.add(Dense(62, activation = "relu"))
model_bidirectional.add(Dropout(0.3))

model_bidirectional.add(Dense(1))
model_bidirectional.compile(optimizer='adam', loss='mse')

model_bidirectional.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model_bidirectional.fit(X_tmp, Y_tmp, epochs=50,
            batch_size=1000,
            validation_split=0.3,
            callbacks=[early_stopping])

/opt/miniconda3/envs/aiao-env/lib/python3.9/site-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/opt/miniconda3/envs/aiao-env/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking_5 (Masking)             │ (None, 72, 9)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 72, 9)          │            36 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 72, 128)        │        37,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 62)             │         4,030 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 62)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            63 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 91,425 (357.13 KB)

 Trainable params: 91,407 (357.06 KB)

 Non-trainable params: 18 (72.00 B)

Epoch 1/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 74s 579ms/step - loss: 2174.9221 - val_loss: 0.0168
Epoch 2/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 71s 575ms/step - loss: 0.0180 - val_loss: 0.0143
Epoch 3/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 75s 602ms/step - loss: 0.0121 - val_loss: 0.0125
Epoch 4/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 76s 613ms/step - loss: 0.0119 - val_loss: 0.0124
Epoch 5/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 75s 602ms/step - loss: 0.0114 - val_loss: 0.0117
Epoch 6/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 72s 582ms/step - loss: 0.0113 - val_loss: 0.0126
Epoch 7/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 75s 603ms/step - loss: 0.0111 - val_loss: 0.0126
Epoch 8/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 72s 579ms/step - loss: 0.0108 - val_loss: 0.0121
Epoch 9/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 74s 600ms/step - loss: 0.0107 - val_loss: 0.0126
Epoch 10/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 74s 595ms/step - loss: 0.0105 - val_loss: 0.0126


In [22]:
Y_pred_bidirectional = model_bidirectional.predict(X_test)
print(Y_pred_bidirectional.shape)
# Y_pred_bidirectional = Y_pred_bidirectional.flatten()
Y_pred_bidirectional = target_scaler.inverse_transform(Y_pred_bidirectional)
print(Y_pred_bidirectional.shape)

print(Y_pred_bidirectional[:, -1])

114/114 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step
(3644, 1)
(3644, 1)
[-1577.9506    -261.378      450.19696  ... -2201.4504      23.670832
    38.089504]


#### Encoder-Decoder

In [23]:
model_LSTM_ED_single = Sequential()
model_LSTM_ED_single.add(Masking(mask_value=-0.01, input_shape=(n_steps, n_features)))
model_LSTM_ED_single.add(BatchNormalization())

model_LSTM_ED_single.add(LSTM(128, activation = "relu", input_shape=(n_steps, n_features)))
model_LSTM_ED_single.add(Dropout(0.3))
model_LSTM_ED_single.add(RepeatVector(1))

model_LSTM_ED_single. add(LSTM(128, activation = "relu", return_sequences=True))
model_LSTM_ED_single.add(Dropout(0.3))

model_LSTM_ED_single.add(TimeDistributed(Dense(100, activation='relu')))
model_LSTM_ED_single.add(TimeDistributed(Dense(1)))

model_LSTM_ED_single.compile(optimizer='adam', loss='mse')
model_LSTM_ED_single.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model_LSTM_ED_single.fit(X_tmp, Y_tmp, epochs=50,
            batch_size=1000,
            validation_split=0.3,
            callbacks=[early_stopping])

/opt/miniconda3/envs/aiao-env/lib/python3.9/site-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/opt/miniconda3/envs/aiao-env/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking_6 (Masking)             │ (None, 72, 9)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 72, 9)          │            36 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_9 (LSTM)                   │ (None, 128)            │        70,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_vector_2 (RepeatVector)  │ (None, 1, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_10 (LSTM)                  │ (None, 1, 128)         │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 1, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_4              │ (None, 1, 100)         │        12,900 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_5              │ (None, 1, 1)           │           101 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 215,277 (840.93 KB)

 Trainable params: 215,259 (840.86 KB)

 Non-trainable params: 18 (72.00 B)

Epoch 1/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 54s 423ms/step - loss: 458.8560 - val_loss: 0.0166
Epoch 2/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 53s 428ms/step - loss: 0.0132 - val_loss: 0.0142
Epoch 3/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 53s 424ms/step - loss: 0.0126 - val_loss: 0.0131
Epoch 4/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 53s 431ms/step - loss: 0.0121 - val_loss: 0.0124
Epoch 5/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 53s 431ms/step - loss: 0.0138 - val_loss: 0.0120
Epoch 6/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 53s 432ms/step - loss: 0.0126 - val_loss: 0.0117
Epoch 7/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 53s 425ms/step - loss: 0.0124 - val_loss: 0.0119
Epoch 8/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 53s 426ms/step - loss: 0.0124 - val_loss: 0.0118
Epoch 9/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 53s 426ms/step - loss: 0.0122 - val_loss: 0.0119
Epoch 10/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 52s 422ms/step - loss: 0.0122 - val_loss: 0.0120
Epoch 11/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 52s 418ms/step - loss: 0.0120 - val_loss: 0.0120


#### CNN-LSTM

In [24]:
model_CNN_LSTM_single = Sequential()
model_CNN_LSTM_single.add(Masking(mask_value=-0.01, input_shape=(n_steps, n_features)))

model_CNN_LSTM_single.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_steps, n_features)))
model_CNN_LSTM_single.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model_CNN_LSTM_single.add(MaxPooling1D(pool_size=2))

model_CNN_LSTM_single.add(Flatten())

model_CNN_LSTM_single.add(RepeatVector(1))
model_CNN_LSTM_single.add(LSTM(128, activation='relu', return_sequences=True))

model_CNN_LSTM_single.add(TimeDistributed(Dense(100, activation='relu')))
model_CNN_LSTM_single.add(TimeDistributed(Dense(1)))

model_CNN_LSTM_single.compile(optimizer='adam', loss='mse')
model_CNN_LSTM_single.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model_CNN_LSTM_single.fit(X_tmp, Y_tmp, epochs=50,
            batch_size=1000,
            validation_split=0.3,
            callbacks=[early_stopping])

/opt/miniconda3/envs/aiao-env/lib/python3.9/site-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/opt/miniconda3/envs/aiao-env/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/opt/miniconda3/envs/aiao-env/lib/python3.9/site-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'conv1d_2' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking_7 (Masking)             │ (None, 72, 9)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 70, 64)         │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 68, 64)         │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 34, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2176)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_vector_3 (RepeatVector)  │ (None, 1, 2176)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_11 (LSTM)                  │ (None, 1, 128)         │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_6              │ (None, 1, 100)         │        12,900 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_7              │ (None, 1, 1)           │           101 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,207,305 (4.61 MB)

 Trainable params: 1,207,305 (4.61 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50


/opt/miniconda3/envs/aiao-env/lib/python3.9/site-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'conv1d_2' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


124/124 ━━━━━━━━━━━━━━━━━━━━ 13s 96ms/step - loss: 0.0249 - val_loss: 0.0118
Epoch 2/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 12s 95ms/step - loss: 0.0118 - val_loss: 0.0112
Epoch 3/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 12s 94ms/step - loss: 0.0107 - val_loss: 0.0105
Epoch 4/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 12s 93ms/step - loss: 0.0103 - val_loss: 0.0107
Epoch 5/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 12s 93ms/step - loss: 0.0098 - val_loss: 0.0103
Epoch 6/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 12s 93ms/step - loss: 0.0097 - val_loss: 0.0105
Epoch 7/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 12s 94ms/step - loss: 0.0094 - val_loss: 0.0107
Epoch 8/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 12s 94ms/step - loss: 0.0091 - val_loss: 0.0104
Epoch 9/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 11s 92ms/step - loss: 0.0088 - val_loss: 0.0107
Epoch 10/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 12s 93ms/step - loss: 0.0087 - val_loss: 0.0121
